In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
# df = pd.read_csv('EOW_DATA_STAGE3_CLEANED.csv')
# df = df.iloc[:, 1:]
# df.head()

In [5]:
X = df.iloc[:, 1:]
y = df['SYM_TOT_COST']

In [6]:
print(X.shape)
print(y.shape)

(38791, 20)
(38791,)


In [8]:
df_cats = df.select_dtypes(include = ['object'])
df_nums = X.select_dtypes(exclude = ['object'])

In [25]:
# df_cats_ohe = pd.get_dummies(df_cats)
# df_cats_ohe.head()

In [10]:
df_cats_ohe.shape

(38791, 116)

In [26]:
# X_ohe = pd.concat([df_cats_ohe, df_nums], axis = 1)
# X_ohe.head()

In [13]:
X_ohe.shape

(38791, 130)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, test_size = 0.2, random_state = 2019)

In [27]:
# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)

In [16]:
import xgboost as xgb
from xgboost import XGBRegressor

mod_1 = XGBRegressor(objective = 'reg:squarederror',
                    n_estimators = 500,
                    random_state = 2019)

In [17]:
param_grid = {'learning_rate': [0.1, 0.2],
             'max_depth': [2,5],
             'subsample': [0.5, 0.8],
             'colsample_bytree': [0.5, 0.8]}

In [18]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator = mod_1,
                   param_grid = param_grid,
                   scoring = 'r2')

In [28]:
# grid.fit(X_train, y_train)

In [20]:
grid.best_params_

{'colsample_bytree': 0.8,
 'learning_rate': 0.1,
 'max_depth': 2,
 'subsample': 0.5}

In [21]:
mod_2 = xgb.XGBRegressor(objective = 'reg:squarederror',
                        booster = 'gbtree',
                        n_estimators = 100,
                        colsample_bytree = 0.8,
                        learning_rate = 0.1,
                        max_depth = 2,
                        subsample = 0.5,
                        random_state = 2019)

In [22]:
eval_set = [(X_train, y_train), (X_test, y_test)]

In [29]:
# mod_2.fit(X_train, y_train,
#          eval_metric = 'rmse',
#          eval_set = eval_set,
#          early_stopping_rounds = 20)

In [30]:
# y_test_pred1 = mod_2.predict(X_test)
# y_test_pred1

In [25]:
dtrain = xgb.DMatrix(data = X_train, label = y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)

C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [26]:
_params = {'colsample_bytree': 0.8,
          'learning_rate': 0.1,
          'max_depth': 2,
          'subsample': 0.5}

In [27]:
watch_list = [(dtrain, 'train'), (dtest, 'test')]

In [31]:
# mod_3 = xgb.train(params = _params,
#                  dtrain = dtrain,
#                  num_boost_round = 500,
#                  evals = watch_list,
#                  early_stopping_rounds = 50)

In [32]:
# y_test_pred2 = mod_3.predict(dtest)
# y_test_pred2

In [33]:
# import numpy as np

# np.corrcoef(y_test_pred1, y_test_pred2)

In [34]:
# np.corrcoef(y_test_pred1, y_test)

In [35]:
# np.corrcoef(y_test, y_test_pred2)

In [33]:
import shap

In [34]:
X_importance = X_test

In [35]:
explainer = shap.TreeExplainer(mod_3)
shap_values = explainer.shap_values(X_importance)

In [36]:
# shap.summary_plot(shap_values, X_importance)

In [37]:
# shap.summary_plot(shap_values, X_importance, plot_type = 'bar')

In [41]:
# X_importance.MAX_FLOOR_SQM.value_counts()

In [42]:
# plt.hist(X_importance.MAX_FLOOR_SQM, bins = 100)

In [43]:
# shap.dependence_plot('MAX_FLOOR_SQM', shap_values, X_importance)

In [44]:
# shap.dependence_plot('ANY_DRYING_IND', shap_values, X_importance)

In [45]:
# shap.dependence_plot('MAX_ROOM_CNT', shap_values, X_importance)

In [44]:
X_interaction = X_importance.iloc[:500, :]

shap_interaction_values = shap.TreeExplainer(mod_3).shap_interaction_values(X_interaction)

In [46]:
# shap.summary_plot(shap_interaction_values, X_interaction)

In [47]:
# shap.dependence_plot('MAX_FLOOR_SQM', shap_values, X_importance)

In [48]:
# shap.dependence_plot(('MAX_FLOOR_SQM', 'MAX_FLOOR_SQM'),
#                     shap_interaction_values,
#                     X_interaction)

In [49]:
# shap.dependence_plot(('MAX_FLOOR_SQM', 'ANY_DRYING_IND'),
#                     shap_interaction_values,
#                     X_interaction)

In [50]:
# shap_sum = np.abs(shap_values).mean(axis = 0)
# importance_df = pd.DataFrame([X_importance.columns.tolist(),
#                              shap_sum.tolist()]).T

# importance_df.columns = ['column_name', 'shap_importance']
# importance_df = importance_df.sort_values('shap_importance', ascending = False)
# importance_df

In [50]:
explainer_2 = shap.TreeExplainer(mod_2)
shap_values_2 = explainer_2.shap_values(X_importance)

In [51]:
# shap.summary_plot(shap_values_2, X_importance)

In [52]:
# df_pred = pd.DataFrame({'SYM_TOT_COST_Test': y_test,
#                        'XGB_1': y_test_pred1,
#                        'XGB_2': y_test_pred2})
# df_pred.head()

In [53]:
# import seaborn as sns

# sns.scatterplot(x = 'SYM_TOT_COST_Test', y = 'XGB_1', data = df_pred)

In [54]:
# sns.scatterplot(x = 'SYM_TOT_COST_Test', y = 'XGB_1', data = df_pred)
# plt.xlim(0, 200000)
# plt.ylim(0, 200000)

In [55]:
# sns.regplot(x = 'SYM_TOT_COST_Test', y = 'XGB_1', data = df_pred)

In [56]:
# sns.regplot(x = 'SYM_TOT_COST_Test', y = 'XGB_1', data = df_pred)
# plt.xlim(0, 200000)
# plt.ylim(0, 200000)

In [57]:
y_pred_all = mod_2.predict(X_ohe)
y_pred_all.shape

(38791,)

In [57]:
# df['Y_Pred_XGB2'] = y_pred_all
# df.head()

In [59]:
df.to_csv('EOW_STAGE3_XGB2_Pred_v2.csv')

In [60]:
from joblib import dump

dump(mod_2, 'xgb2_2.joblib')
dump(mod_3, 'xgb3_2.joblib')

['xgb3_2.joblib']

In [58]:
# p1 = sns.distplot(df_pred['SYM_TOT_COST_Test'], color = 'r', hist = False, label = 'True')
# p2 = sns.distplot(df_pred['XGB_1'], color = 'b', hist = False, label = 'XGB_1')
# p3 = sns.distplot(df_pred['XGB_2'], color = 'y', hist = False, label = 'XGB_2')
# plt.xlim(0, 25000)

In [59]:
# p1 = sns.distplot(df_pred['SYM_TOT_COST_Test'], color = 'r', hist =False, label = 'True')
# p2 = sns.distplot(df_pred['XGB_1'], color = 'b', hist = False, label = 'XGB_1')
# p3 = sns.distplot(df_pred['XGB_2'], color = 'y', hist = False, label = 'XGB_2')
# plt.xlim(0, 10000)

In [60]:
# p1 = sns.distplot(df_pred['SYM_TOT_COST_Test'], color = 'r', hist = False, label = 'True')
# p2 = sns.distplot(df_pred['XGB_1'], color = 'b', hist = False, label = 'XGB_1')
# p3 = sns.distplot(df_pred['XGB_2'], color = 'y', hist = False, label = 'XGB_2')

In [61]:
# df_pred['SYM_TOT_COST_Test'].max()

In [62]:
# df_pred['XGB_2'].max()

In [63]:
# df_pred['XGB_1'].max()

In [64]:
# p1 = sns.distplot(df_pred['SYM_TOT_COST_Test'], color = 'r', hist = False, label = 'True')
# p2 = sns.distplot(df_pred['XGB_1'], color = 'b', hist = False, label = 'XGB_1')
# p3 = sns.distplot(df_pred['XGB_2'], color = 'y', hist = False, label = 'XGB_2')
# plt.xlim(0, 100000)